In [ ]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size = 0.2, random_state = 421)

sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size = 0.2, random_state = 421)

In [ ]:
sub_input.shape, val_input.shape

((4157, 3), (1040, 3))

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

DecisionTreeClassifier(random_state=42)

In [ ]:
dt.score(sub_input, sub_target),dt.score(val_input, val_target)

(0.997834977146981, 0.85)

In [ ]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)

In [ ]:
scores

{'fit_time': array([0.01950526, 0.02030849, 0.01130271, 0.01257229, 0.01132536]),
 'score_time': array([0.00969505, 0.00188279, 0.00177312, 0.00255609, 0.00196743]),
 'test_score': array([0.84615385, 0.85865385, 0.84311838, 0.85851781, 0.84600577])}

In [ ]:
import numpy as np
np.mean(scores['test_score'])

0.8504899311468128

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())
np.mean(scores['test_score'])

0.8504899311468128

In [ ]:
splitter = StratifiedKFold(n_splits = 10, shuffle=True, random_state = 42)
scores = cross_validate(dt, train_input, train_target, cv = splitter)
np.mean(scores['test_score'])

0.8535701052319549

In [ ]:
from sklearn.model_selection import GridSearchCV

param = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), param, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [ ]:
dt = gs.best_estimator_
dt.score(train_input, train_target)

0.9047527419665191

In [ ]:
gs.best_params_

{'min_impurity_decrease': 0.0003}

In [ ]:
gs.cv_results_['mean_test_score']

array([0.8572244 , 0.86530651, 0.86550141, 0.86376897, 0.86241949])

In [ ]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001), 'max_depth' : range(5, 20, 1), 'min_samples_split' : range(2, 100, 10)}

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [ ]:
dt = gs.best_estimator_
dt.score(train_input, train_target)

0.9274581489320762

In [ ]:
gs.best_params_

{'max_depth': 17, 'min_impurity_decrease': 0.0002, 'min_samples_split': 2}

In [ ]:
np.max(gs.cv_results_['mean_test_score'])

0.8662698970903975

In [ ]:
from scipy.stats import uniform, randint

In [ ]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001), 'max_depth' : randint(20, 50), 'min_samples_leaf' : randint(1, 25)}
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7e4be6df3ca0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7e4be7b62f20>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7e4be6df3550>},
                   random_state=42)

In [ ]:
gs.best_params_

{'max_depth': 20,
 'min_impurity_decrease': 0.0003395618906669724,
 'min_samples_leaf': 2}